This notebook compares ZNE noise scaling methods: identity insertion and unitary folding.

Unitary folding: $G \mapsto GG^\dag G$ \
Identity insertion: $G \mapsto IG$

In [1]:
from mitiq.benchmarks import generate_ghz_circuit
from mitiq.zne.scaling import insert_id_layers, fold_global

demo = generate_ghz_circuit(3)
scale_factor = 3

print("-----ORIGINAL-----")
print(demo)
print("\n-----------------FOLDING------------------")
print(fold_global(demo, scale_factor))
print("\n-----------------SCALING------------------")
print(insert_id_layers(demo, scale_factor))

-----ORIGINAL-----
0: ───H───@───────
          │
1: ───────X───@───
              │
2: ───────────X───

-----------------FOLDING------------------
0: ───H───@───────────@───H───H───@───────
          │           │           │
1: ───────X───@───@───X───────────X───@───
              │   │                   │
2: ───────────X───X───────────────────X───

-----------------SCALING------------------
0: ───H───I───I───@───I───I───────I───I───
                  │
1: ───────I───I───X───I───I───@───I───I───
                              │
2: ───────I───I───────I───I───X───I───I───


### WARNING
Unitary folding scales noise by applying an additional layer $GG^\dag$ to the circuit. For non-hermitian gates $G$ and $G^\dag$ may not have the same noise model, and hence noise is potentially scaled in an non-linear way.

Similarly, the noise that predominantly scaled in identity insertion is that of idle qubit noise/decoherence.

In [2]:
print(
    "{: >12}  {: ^14} {: ^14} {: ^15}".format(
        "scale factor", "original depth", "folded depth", "id insertion depth"
    )
)
for scale_factor in range(1, 10):
    folded_depth = len(fold_global(demo, scale_factor))
    id_insert_depth = len(insert_id_layers(demo, scale_factor))
    print(
        "{: >12}  {: ^14} {: ^14} {: ^15}".format(
            scale_factor, len(demo), folded_depth, id_insert_depth
        )
    )

scale factor  original depth  folded depth  id insertion depth
           1        3              3               3       
           2        3              7               6       
           3        3              9               9       
           4        3              13             12       
           5        3              15             15       
           6        3              19             18       
           7        3              21             21       
           8        3              25             24       
           9        3              27             27       


In [3]:
from mitiq.zne import execute_with_zne
import cirq

def execute(circuit, noise_level=0.05):
    noisy_circuit = circuit.with_noise(cirq.depolarize(p=noise_level))
    return (
        cirq.DensityMatrixSimulator()
        .simulate(noisy_circuit)
        .final_density_matrix[0, 0]
        .real
    )

In [4]:
execute_with_zne(generate_ghz_circuit(6), execute, scale_noise=insert_id_layers)

0.32638816162943873

In [5]:
def variational_circuit(gamma):
    q0, q1 = cirq.LineQubit.range(2)

    return cirq.Circuit(
        [
            cirq.rx(gamma)(q0),
            cirq.CNOT(q0, q1),
            cirq.rx(gamma)(q1),
            cirq.CNOT(q0, q1),
            cirq.rx(gamma)(q0),
        ]
    )

In [6]:
from random import uniform
import numpy as np

results = {"fold": [], "id": []}
for _ in range(100):
    gamma = uniform(0, 2 * np.pi)
    circuit = variational_circuit(gamma)

    ideal_expval = execute(circuit, noise_level=0.0)
    noisy_expval = execute(circuit)
    folded_expval = execute_with_zne(circuit, execute, scale_noise=fold_global)
    id_expval = execute_with_zne(circuit, execute, scale_noise=insert_id_layers)

    noisy_error = abs(ideal_expval - noisy_expval)
    folded_IF = noisy_error / abs(ideal_expval - folded_expval)
    scaled_IF = noisy_error / abs(ideal_expval - id_expval)

    results["fold"].append(folded_IF)
    results["id"].append(scaled_IF)

print("Avg improvement factor (`fold_global`):      ", round(np.average(results["fold"]), 4))
print("Avg improvement factor (`insert_id_layers`): ", round(np.average(results["id"]), 4))

Avg improvement factor (`fold_global`):       1.8483
Avg improvement factor (`insert_id_layers`):  7.9908
